In [57]:
%load_ext autoreload
%autoreload 2
from src.VLLMServer import launch_command,get_client_dict,ask_llm,ask_llm_prompt
from src.code_utils import extract_code_block
from src.process_openmath_ja import parse_record,eval_answer
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
conf_path="config.yaml"

with open(conf_path,"r") as f:
    conf=yaml.safe_load(f.read())
client_dict=get_client_dict(conf)

print("vllmサーバーの実行コマンド")
print(launch_command(conf))

#client_dict

vllmサーバーの実行コマンド
export CUDA_VISIBLE_DEVICES=1
python -m vllm.entrypoints.openai.api_server --model microsoft/Phi-3-medium-128k-instruct --max-model-len 4000 --port 8001 --gpu-memory-utilization 0.4 




In [68]:
model_name="microsoft/Phi-3-medium-128k-instruct"
pre_command="""次の問題を解きなさい｡出力事項は以下のとおりである｡
#解説: 問題を解くための基本的な考え方を日本語で出力する｡
#Python: 計算プログラム｡以下の例を参考に､出力形式を守ること｡
```
#a+bの計算を行う
a=1
b=2
print(a+b)
```
#問題
"""

In [65]:
#load dataset
from datasets import load_dataset
ds=load_dataset("kunishou/OpenMathInstruct-1-1.8m-ja",split="train",streaming=False)

In [67]:
ds=ds.shuffle()

In [90]:

record=ds[0]

#問題文と回答の取得
question,answer=parse_record(record)

#promptの生成
prompt=pre_command+question
res=ask_llm_prompt(client_dict,model_name,prompt)

In [91]:
#生成されたコードの評価
explanation_block=res[:res.find("```")]
code_block=extract_code_block(res)

#真偽判定
is_correct_answer=eval_answer(code_block,answer,verbose=False)

generated_prompt=explanation_block+"\n"+code_block
if is_correct_answer:
    print(generated_prompt)

#print(explanation_block)
#print(code_block)

#解説:
直交座標系における2点間の距離を求めるには、距離公式による計算が用いられます。距離公式は以下の通りです。

$$d = \sqrt{(x2 - x1)^2 + (y2 - y1)^2}$$

原点 (0, 0) から点 (-15, 8) までの距離を求めます。

#Python:

import math

# 座標系の原点
x1, y1 = 0, 0

# 指定された点
x2, y2 = -15, 8

# 距離計算
distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# 距離を出力
print(distance)


In [83]:
code_block

'#原点から点(-15,8)までの距離を求める\na = -15  #水平方向の距離(原点から左に15単位移動)\nb = 8    #垂直方向の距離(原点から上に8単位移動)\n\n#ピタゴラスの定理を用いて距離を求める\ndistance = (a**2 + b**2) ** 0.5\nprint(distance)'

'29.0\n'